-- cosshubtest.university definition

CREATE TABLE `university` (
  `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',
  `univ_name_arlimi` varchar(100) DEFAULT NULL COMMENT '대학명(알리미기준)',
  `univ_name` varchar(100) NOT NULL COMMENT '대학명(일반명칭)',
  `univ_name_en` varchar(100) NOT NULL COMMENT '대학명(영문)',
  `univ_type` varchar(50) NOT NULL COMMENT '학교종류',
  `establish` varchar(10) NOT NULL COMMENT '설립구분',
  `status` varchar(10) NOT NULL COMMENT '상태',
  `campus` varchar(10) NOT NULL COMMENT '캠퍼스',
  `campus_name` varchar(50) NOT NULL COMMENT '캠퍼스명',
  `locale` varchar(10) NOT NULL COMMENT '지역',
  `homepage` varchar(50) NOT NULL COMMENT '학교 홈페이지',
  `tel` varchar(20) NOT NULL COMMENT '전화번호',
  `fax` varchar(20) NOT NULL COMMENT '팩스번호',
  PRIMARY KEY (`univ_cd`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='대학교정보';

In [ ]:
# 설치되지 않은 라이브러리의 경우, 주석 해제 후 코드를 실행하여 설치
#!pip install pandas
#%pip install pandas

# Pandas
#%pip install pandas
import pandas as pd

In [ ]:
# DB 접속 정보
ip = '133.186.247.101'
id = 'root'
pwd = 'coss!@#$'

dbname = 'cosshubtest'
table = 'university'

excel_path = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/05.설계/01.테이블설계/01.Codes.xlsx'
sheet_name = '학교코드정보'

In [ ]:
#%pip install pymysql
import pymysql

conn = pymysql.connect(host=ip, user=id, password=pwd, db=dbname)
curs = conn.cursor(pymysql.cursors.DictCursor)

In [ ]:
# excel file load
#%pip install openpyxl
df = pd.read_excel(excel_path, sheet_name = sheet_name)


In [ ]:
# 데이터프레임의 nan 값을 None으로 변경
#df.dropna(inplace=True)
#df = df.where((pd.notnull(df)), 'None')

df = df.fillna('None')

In [ ]:
df

In [ ]:
#df.rename(columns={'대학 고유코드':'a'}, inplace=True) #특정 컬럼명 변경
#df.columns('a', 'b') 


cols_ko = list(df.columns) #컬럼 시리즈를 리스트로 변경
cols_en = list(df.iloc[0]) #데이터 시리즈를 리스트로 변경

for n in range(len(cols_ko)):
    #print(cols_en[n], cols_ko[n])
    df.rename(columns={cols_ko[n]:cols_en[n]}, inplace=True) #특정 컬럼명 변경
df = df.drop(0) #특정행 삭제

df

In [ ]:
# db truncate tables
# 기존 데이터를 삭제 후 인서트, 계속 추가하는 로직이면 삭제
sql_truncate = 'truncate table {}.{}'.format(dbname, table)

curs.execute(sql_truncate)
conn.commit()


In [ ]:
# data insert to db

field_names = ''
field_data = ''

for n in range(len(cols_en)):
    if n == 0 :
        field_names = cols_en[n]
        field_data = '%s'
    else:
        field_names = field_names + ', ' + cols_en[n]
        field_data = field_data + ', ' + '%s'
        
        
# sql_insert = 'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
# print(sql_insert)

sql_insert = 'insert into {}.{}({}) values({})'.format(dbname, table, field_names, field_data)
#print(sql)
for idx in range(len(df)):
    	curs.execute(sql_insert, tuple(df.values[idx]))
conn.commit()


# 중도탈락 현황을 DB에 등록
4-사. 중도탈락 학생 현황.xlsx + 4-바-2. 외국학생 중도탈락 현황.xlsx 을 통합

-- cosshubtest.dropout_stat definition

CREATE TABLE `dropout_stat` (
  `year` int(4) NOT NULL COMMENT '기준연도',
  `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',
  `stud_tot` int(5) DEFAULT 0 COMMENT '재적_학생수',
  `stud_sum` int(5) DEFAULT 0 COMMENT '재적_중도탈락_합계',
  `stud_unregist` int(5) DEFAULT 0 COMMENT '재적_중도탈락_미등록',
  `stud_unreturn` int(5) DEFAULT 0 COMMENT '재적_중도탈락_미복학',
  `stud_drop` int(5) DEFAULT 0 COMMENT '재적_중도탈락_자퇴',
  `stud_warn` int(5) DEFAULT 0 COMMENT '재적_중도탈락_학사경고',
  `stud_act` int(5) DEFAULT 0 COMMENT '재적_중도탈락_학생활동',
  `stud_expulsion` int(5) DEFAULT 0 COMMENT '재적_중도탈락_유급제적',
  `stud_attendyear` int(5) DEFAULT 0 COMMENT '재적_중도탈락_재학연한초과',
  `stud_etc` int(5) DEFAULT 0 COMMENT '재적_중도탈락_기타',
  `stud_rate` float(5,2) DEFAULT 0.00 COMMENT '재적_중도탈락_비율',
  `fresh_tot` int(5) DEFAULT 0 COMMENT '신입_학생수',
  `fresh_sum` int(5) DEFAULT 0 COMMENT '신입_중도탈락_합계',
  `fresh_unregist` int(5) DEFAULT 0 COMMENT '신입_중도탈락_미등록',
  `fresh_unreturn` int(5) DEFAULT 0 COMMENT '신입_중도탈락_미복학',
  `fresh_drop` int(5) DEFAULT 0 COMMENT '신입_중도탈락_자퇴',
  `fresh_warn` int(5) DEFAULT 0 COMMENT '신입_중도탈락_학사경고',
  `fresh_act` int(5) DEFAULT 0 COMMENT '신입_중도탈락_학생활동',
  `fresh_expulsion` int(5) DEFAULT 0 COMMENT '신입_중도탈락_유급제적',
  `fresh_attendyear` int(5) DEFAULT 0 COMMENT '신입_중도탈락_재학연한초과',
  `fresh_etc` int(5) DEFAULT 0 COMMENT '신입_중도탈락_기타',
  `fresh_rate` float(5,2) DEFAULT 0.00 COMMENT '신입_중도탈락_비율',
  `foreign_tot` int(5) DEFAULT 0 COMMENT '외국인_학생수',
  `foreign_sum` int(5) DEFAULT 0 COMMENT '외국인_중도탈락_합계',
  `foreign_degree` int(5) DEFAULT 0 COMMENT '외국인_학위과정(대학)',
  `foreign_common` int(5) DEFAULT 0 COMMENT '외국인_교육과정공동운영생',
  `foreign_lang` int(5) DEFAULT 0 COMMENT '외국인_어학연수생',
  `foreign_exchange` int(5) DEFAULT 0 COMMENT '외국인_교환학생',
  `foreign_visit` int(5) DEFAULT 0 COMMENT '외국인_방문학생',
  `foreign_etc` int(5) DEFAULT 0 COMMENT '외국인_기타연수생',
  `foreign_rate` float(5,2) DEFAULT 0.00 COMMENT '외국인_중도탈락_비율',
  PRIMARY KEY (`year`,`univ_cd`),
  KEY `univ_cd` (`univ_cd`),
  CONSTRAINT `dropout_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='중도탈락 학생 현황';


## 중도탈락 엑셀을 DB로 인서트
- 한국 학생 + 외국학생 두개의 파일
- 기준연도	학교종류	설립구분	지역	상태	학교	재적학생\n(A)	사유별 중도탈락 학생 계(B)	미등록	미복학	...	사유별 중도탈락 학생(신입생) 계(B`)	미등록	미복학	자퇴	학사경고	학생활동	유급제적	재학연한초과	기타	중도탈락학생(신입생)비율(%)\n(B`/A`) × 100
- 기준연도	학교종류	설립구분	지역	상태	학교	외국 재적학생 총계(A)	학위과정(대학)	교육과정공동운영생	어학연수생	...	방문학생	기타연수생	중도탈락 학생 총계(B)	학위과정(대학)	교육과정공동운영생	어학연수생	교환학생	방문학생	기타연수생	외국 학생 중도탈락률(%)(B/A)×100



In [784]:
# 설치되지 않은 라이브러리의 경우, 주석 해제 후 코드를 실행하여 설치
#!pip install pandas
#%pip install pandas

# Pandas
#%pip install pandas
import pandas as pd


# DB 접속 정보
ip = '133.186.247.101'
id = 'root'
pwd = 'coss!@#$'

dbname = 'cosshubtest'  ## 변경 필요
table_univ = 'university'    ## 변경 필요
table_dropout = 'dropout_stat'    ## 변경 필요


excel_path1 = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/1.학생/4-사. 중도탈락 학생 현황 - 2022.xlsx'
sheet_name1 = '원본'

excel_path2 = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/1.학생/4-바-2. 외국학생 중도탈락 현황 - 2022.xlsx'
sheet_name2 = '원본'


#%pip install pymysql
import pymysql

conn = pymysql.connect(host=ip, user=id, password=pwd, db=dbname)
curs = conn.cursor(pymysql.cursors.DictCursor)

if conn == None:
    print('DB 연결 오류')
elif curs == None:
    print('curs 연결 오류')


# excel file load
#%pip install openpyxl
df1 = pd.read_excel(excel_path1, sheet_name = sheet_name1)
df2 = pd.read_excel(excel_path2, sheet_name = sheet_name2)


print(excel_path1)
print(excel_path2)

D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/1.학생/4-사. 중도탈락 학생 현황 - 2022.xlsx
D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/1.학생/4-바-2. 외국학생 중도탈락 현황 - 2022.xlsx


In [785]:
##########################################################################
# 중도탈락 학생 현황 필드 변경 (0번째와 1번째 데이터 행을 타이틀행으로 변경)
# 엑셀의 제목행은 기본 두줄이므로 두줄을 통합해야함

# 2023년_ [4-사. 중도탈락 학생 현황]
# 제목0
# 제목1
##########################################################################

#df.rename(columns={'대학 고유코드':'a'}, inplace=True) #특정 컬럼명 변경
#df.columns('a', 'b') 

cols_field = list(df1.columns) #컬럼 시리즈를 리스트로 변경
cols_data0 = list(df1.iloc[0]) #데이터0 시리즈를 리스트로 변경
cols_data1 = list(df1.iloc[1]) #데이터1 시리즈를 리스트로 변경

for n in range(len(cols_field)):
    if pd.isna(cols_data0[n]) == True and pd.isna(cols_data1[n]) == False:
        cols_data0[n] = cols_data1[n]
        
    elif pd.isna(cols_data0[n]) == False and pd.isna(cols_data1[n]) == False:
        cols_data0[n] = cols_data0[n] + ' ' + cols_data1[n]        
    
for n in range(len(cols_field)):
    df1.rename(columns={cols_field[n]:cols_data0[n]}, inplace=True) #특정 컬럼명 변경
    
df1 = df1.drop(0) #제목 3행중 2행 삭제
df1 = df1.drop(1) #제목 3행중 2행 삭제

##########################################################################
cols_field = list(df2.columns) #컬럼 시리즈를 리스트로 변경
cols_data0 = list(df2.iloc[0]) #데이터0 시리즈를 리스트로 변경
cols_data1 = list(df2.iloc[1]) #데이터1 시리즈를 리스트로 변경

for n in range(len(cols_field)):
    if pd.isna(cols_data0[n]) == True and pd.isna(cols_data1[n]) == False:
        cols_data0[n] = cols_data1[n]
        
    elif pd.isna(cols_data0[n]) == False and pd.isna(cols_data1[n]) == False:
        cols_data0[n] = cols_data0[n] + ' ' + cols_data1[n]        
    
for n in range(len(cols_field)):
    df2.rename(columns={cols_field[n]:cols_data0[n]}, inplace=True) #특정 컬럼명 변경
    
df2 = df2.drop(0) #제목 3행중 2행 삭제
df2 = df2.drop(1) #제목 3행중 2행 삭제


In [786]:
# 데이터프레임의 nan 값을 None으로 변경
#df.dropna(inplace=True)
#df = df.where((pd.notnull(df)), 'None')

df1 = df1.fillna(0)
df2 = df2.fillna(0)

In [787]:
df1

,기준연도,학교종류,설립구분,지역,상태,학교,재적학생\n(A),사유별 중도탈락 학생 계(B),미등록,미복학,...,사유별 중도탈락 학생(신입생) 계(B`),미등록,미복학,자퇴,학사경고,학생활동,유급제적,재학연한초과,기타,중도탈락학생(신입생)비율(%)\n(B`/A`) × 100
2,2022,대학교,사립,경남,기존,가야대학교(김해),2355,218,29,50,...,36,6,0,30,0,0,0,0,0,9.40
3,2022,대학교,사립,경기,기존,가천대학교,26126,945,125,162,...,330,10,0,319,0,0,0,0,1,7.06
4,2022,대학교,사립,강원,기존,가톨릭관동대학교,10959,850,318,0,...,157,3,0,153,0,0,0,0,1,10.04
5,2022,대학교,사립,충북,기존,가톨릭꽃동네대학교,545,24,5,0,...,7,0,0,7,0,0,0,0,0,6.48
6,2022,대학교,사립,서울,기존,가톨릭대학교 _제3캠퍼스,306,6,1,0,...,1,1,0,0,0,0,0,0,0,2.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,2022,산업대학,사립,전북,기존,호원대학교,6644,745,209,177,...,211,111,0,100,0,0,0,0,0,15.33
242,2022,대학교,사립,세종,기존,홍익대학교 _제2캠퍼스,7961,441,68,64,...,180,39,0,141,0,0,0,0,0,13.02
243,2022,대학교,사립,서울,기존,홍익대학교,17235,628,67,70,...,281,28,0,253,0,0,0,0,0,9.77
244,2022,대학교,사립,경기,기존,화성의과학대학교,586,53,4,15,...,10,0,0,10,0,0,0,0,0,14.08


In [788]:
df2

,기준연도,학교종류,설립구분,지역,상태,학교,외국 재적학생 총계(A),학위과정(대학),교육과정공동운영생,어학연수생,...,방문학생,기타연수생,중도탈락 학생 총계(B),학위과정(대학),교육과정공동운영생,어학연수생,교환학생,방문학생,기타연수생,외국 학생 중도탈락률(%)(B/A)×100
2,2022,대학교,사립,경남,기존,가야대학교(김해),21,21,0,0,...,0,0,2,2,0,0,0,0,0,9.5
3,2022,대학교,사립,경기,기존,가천대학교,2131,1928,0,156,...,0,0,140,140,0,0,0,0,0,6.6
4,2022,대학교,사립,강원,기존,가톨릭관동대학교,49,48,1,0,...,0,0,6,5,0,1,0,0,0,12.2
5,2022,대학교,사립,경기,기존,가톨릭대학교,499,361,0,88,...,0,0,22,22,0,0,0,0,0,4.4
6,2022,대학교,사립,서울,기존,가톨릭대학교 _제2캠퍼스,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2022,대학교,사립,충남,기존,호서대학교,355,288,0,63,...,0,0,34,22,0,12,0,0,0,9.6
220,2022,산업대학,사립,전북,기존,호원대학교,173,97,0,76,...,0,0,17,17,0,0,0,0,0,9.8
221,2022,대학교,사립,서울,기존,홍익대학교,1668,1177,0,412,...,6,0,115,75,0,40,0,0,0,6.9
222,2022,대학교,사립,세종,기존,홍익대학교 _제2캠퍼스,110,110,0,0,...,0,0,21,21,0,0,0,0,0,19.1


In [789]:
df2.columns[12]

'기타연수생'

In [790]:
# df2의 열 삭제를 위해 열 이름 변경

df2_del = df2.columns.tolist()

print(type(df2_del), len(df2_del), df2_del)

df2_del[7] = 'DEL'
df2_del[8] = 'DEL'
df2_del[9] = 'DEL'
df2_del[10] = 'DEL'
df2_del[11] = 'DEL'
df2_del[12] = 'DEL'


# 전체 열 이름 입력하기
df2.columns = df2_del

# 선택하여 열 이름 변경하기
#df.rename(columns={'Before':'After'})

print(type(df2_del), len(df2_del), df2_del)
print(df2.columns)


<class 'list'> 21 ['기준연도', '학교종류', '설립구분', '지역', '상태', '학교', '외국 재적학생 총계(A)', '학위과정(대학)', '교육과정공동운영생', '어학연수생', '교환학생', '방문학생', '기타연수생', '중도탈락 학생 총계(B)', '학위과정(대학)', '교육과정공동운영생', '어학연수생', '교환학생', '방문학생', '기타연수생', '외국 학생 중도탈락률(%)(B/A)×100']
<class 'list'> 21 ['기준연도', '학교종류', '설립구분', '지역', '상태', '학교', '외국 재적학생 총계(A)', 'DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'DEL', '중도탈락 학생 총계(B)', '학위과정(대학)', '교육과정공동운영생', '어학연수생', '교환학생', '방문학생', '기타연수생', '외국 학생 중도탈락률(%)(B/A)×100']
Index(['기준연도', '학교종류', '설립구분', '지역', '상태', '학교', '외국 재적학생 총계(A)', 'DEL', 'DEL',
       'DEL', 'DEL', 'DEL', 'DEL', '중도탈락 학생 총계(B)', '학위과정(대학)', '교육과정공동운영생',
       '어학연수생', '교환학생', '방문학생', '기타연수생', '외국 학생 중도탈락률(%)(B/A)×100'],
      dtype='object')


In [791]:
# 중복 또는 불필요한 df의 열 삭제

df1 = df1.drop(['설립구분'], axis = 1)
df1 = df1.drop(['학교종류'], axis = 1)
df1 = df1.drop(['지역'], axis = 1)
df1 = df1.drop(['상태'], axis = 1)


#df2 = df2.drop(df2.columns[7], axis=1) # 학위과정(대학) 인데 동일한 컬럼명이 모두 삭제됨


df2 = df2.drop(['설립구분'], axis = 1)
df2 = df2.drop(['학교종류'], axis = 1)
df2 = df2.drop(['지역'], axis = 1)
df2 = df2.drop(['상태'], axis = 1)

df2 = df2.drop(['DEL'], axis = 1)



# df2 = df2.drop(['학위과정(대학)'], axis = 1)
# df2 = df2.drop(['교육과정공동운영생'], axis = 1)
# df2 = df2.drop(['어학연수생'], axis = 1)
# df2 = df2.drop(['교환학생'], axis = 1)
# df2 = df2.drop(['방문학생'], axis = 1)
# df2 = df2.drop(['기타연수생'], axis = 1)


In [792]:
df1

,기준연도,학교,재적학생\n(A),사유별 중도탈락 학생 계(B),미등록,미복학,자퇴,학사경고,학생활동,유급제적,...,사유별 중도탈락 학생(신입생) 계(B`),미등록,미복학,자퇴,학사경고,학생활동,유급제적,재학연한초과,기타,중도탈락학생(신입생)비율(%)\n(B`/A`) × 100
2,2022,가야대학교(김해),2355,218,29,50,139,0,0,0,...,36,6,0,30,0,0,0,0,0,9.40
3,2022,가천대학교,26126,945,125,162,650,0,0,0,...,330,10,0,319,0,0,0,0,1,7.06
4,2022,가톨릭관동대학교,10959,850,318,0,531,0,0,0,...,157,3,0,153,0,0,0,0,1,10.04
5,2022,가톨릭꽃동네대학교,545,24,5,0,19,0,0,0,...,7,0,0,7,0,0,0,0,0,6.48
6,2022,가톨릭대학교 _제3캠퍼스,306,6,1,0,4,1,0,0,...,1,1,0,0,0,0,0,0,0,2.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,2022,호원대학교,6644,745,209,177,358,0,0,0,...,211,111,0,100,0,0,0,0,0,15.33
242,2022,홍익대학교 _제2캠퍼스,7961,441,68,64,288,11,0,0,...,180,39,0,141,0,0,0,0,0,13.02
243,2022,홍익대학교,17235,628,67,70,434,35,0,0,...,281,28,0,253,0,0,0,0,0,9.77
244,2022,화성의과학대학교,586,53,4,15,34,0,0,0,...,10,0,0,10,0,0,0,0,0,14.08


In [793]:
df2

,기준연도,학교,외국 재적학생 총계(A),중도탈락 학생 총계(B),학위과정(대학),교육과정공동운영생,어학연수생,교환학생,방문학생,기타연수생,외국 학생 중도탈락률(%)(B/A)×100
2,2022,가야대학교(김해),21,2,2,0,0,0,0,0,9.5
3,2022,가천대학교,2131,140,140,0,0,0,0,0,6.6
4,2022,가톨릭관동대학교,49,6,5,0,1,0,0,0,12.2
5,2022,가톨릭대학교,499,22,22,0,0,0,0,0,4.4
6,2022,가톨릭대학교 _제2캠퍼스,1,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
219,2022,호서대학교,355,34,22,0,12,0,0,0,9.6
220,2022,호원대학교,173,17,17,0,0,0,0,0,9.8
221,2022,홍익대학교,1668,115,75,0,40,0,0,0,6.9
222,2022,홍익대학교 _제2캠퍼스,110,21,21,0,0,0,0,0,19.1


## 두개의 파일을 두개의 키 값을 기준으로 결합

### 1. df1의 컬럼에 df2에서 가져올 컬럼으로 추가함
### 2. 연도+학교명 키값으로 두개의 df를 결합

In [794]:
# 1. df1의 컬럼에 df2에서 가져올 컬럼으로 추가함

cols_field1 = list(df1.columns) #컬럼 시리즈를 리스트로 변경
cols_field2 = list(df2.columns) #컬럼 시리즈를 리스트로 변경

In [795]:
print(cols_field1)
print(cols_field2)


['기준연도', '학교', '재적학생\n(A)', '사유별 중도탈락 학생 계(B)', '미등록', '미복학', '자퇴', '학사경고', '학생활동', '유급제적', '재학연한초과', '기타', '중도탈락학생비율(%)\n(B/A) × 100', '재적학생\n(신입생)\n(A`)', '사유별 중도탈락 학생(신입생) 계(B`)', '미등록', '미복학', '자퇴', '학사경고', '학생활동', '유급제적', '재학연한초과', '기타', '중도탈락학생(신입생)비율(%)\n(B`/A`) × 100']
['기준연도', '학교', '외국 재적학생 총계(A)', '중도탈락 학생 총계(B)', '학위과정(대학)', '교육과정공동운영생', '어학연수생', '교환학생', '방문학생', '기타연수생', '외국 학생 중도탈락률(%)(B/A)×100']


In [796]:
# 외국인 중도탈락 필드를 수동으로 지정
cols_add = [ cols_field2[2], '외국 ' + cols_field2[3], '외국 ' + cols_field2[4], '외국 ' + cols_field2[5], '외국 ' + cols_field2[6], '외국 ' + cols_field2[7], '외국 ' + cols_field2[8], '외국 ' + cols_field2[9], cols_field2[10] ]

print(cols_add)

['외국 재적학생 총계(A)', '외국 중도탈락 학생 총계(B)', '외국 학위과정(대학)', '외국 교육과정공동운영생', '외국 어학연수생', '외국 교환학생', '외국 방문학생', '외국 기타연수생', '외국 학생 중도탈락률(%)(B/A)×100']


In [797]:
#df1 = df1.reindex(columns=df1.columns.to_list() + [ '외국' + cols_field2[6], '외국' + cols_field2[13], '외국' + cols_field2[14], '외국' + cols_field2[15], '외국' + cols_field2[16], '외국' + cols_field2[17], '외국' + cols_field2[18], '외국' + cols_field2[19], '외국' + cols_field2[20] ])

for n in range(len(cols_add)): #추가할 필드만큼만 루프
    print(n, len(cols_field1)+n, cols_add[n])
    df1.insert(len(cols_field1)+n, cols_add[n], 0)

0 24 외국 재적학생 총계(A)
1 25 외국 중도탈락 학생 총계(B)
2 26 외국 학위과정(대학)
3 27 외국 교육과정공동운영생
4 28 외국 어학연수생
5 29 외국 교환학생
6 30 외국 방문학생
7 31 외국 기타연수생
8 32 외국 학생 중도탈락률(%)(B/A)×100


In [798]:
df1.columns

Index(['기준연도', '학교', '재적학생\n(A)', '사유별 중도탈락 학생 계(B)', '미등록', '미복학', '자퇴',
       '학사경고', '학생활동', '유급제적', '재학연한초과', '기타', '중도탈락학생비율(%)\n(B/A) × 100',
       '재적학생\n(신입생)\n(A`)', '사유별 중도탈락 학생(신입생) 계(B`)', '미등록', '미복학', '자퇴',
       '학사경고', '학생활동', '유급제적', '재학연한초과', '기타',
       '중도탈락학생(신입생)비율(%)\n(B`/A`) × 100', '외국 재적학생 총계(A)', '외국 중도탈락 학생 총계(B)',
       '외국 학위과정(대학)', '외국 교육과정공동운영생', '외국 어학연수생', '외국 교환학생', '외국 방문학생',
       '외국 기타연수생', '외국 학생 중도탈락률(%)(B/A)×100'],
      dtype='object')

In [799]:
df1

,기준연도,학교,재적학생\n(A),사유별 중도탈락 학생 계(B),미등록,미복학,자퇴,학사경고,학생활동,유급제적,...,중도탈락학생(신입생)비율(%)\n(B`/A`) × 100,외국 재적학생 총계(A),외국 중도탈락 학생 총계(B),외국 학위과정(대학),외국 교육과정공동운영생,외국 어학연수생,외국 교환학생,외국 방문학생,외국 기타연수생,외국 학생 중도탈락률(%)(B/A)×100
2,2022,가야대학교(김해),2355,218,29,50,139,0,0,0,...,9.40,0,0,0,0,0,0,0,0,0
3,2022,가천대학교,26126,945,125,162,650,0,0,0,...,7.06,0,0,0,0,0,0,0,0,0
4,2022,가톨릭관동대학교,10959,850,318,0,531,0,0,0,...,10.04,0,0,0,0,0,0,0,0,0
5,2022,가톨릭꽃동네대학교,545,24,5,0,19,0,0,0,...,6.48,0,0,0,0,0,0,0,0,0
6,2022,가톨릭대학교 _제3캠퍼스,306,6,1,0,4,1,0,0,...,2.04,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,2022,호원대학교,6644,745,209,177,358,0,0,0,...,15.33,0,0,0,0,0,0,0,0,0
242,2022,홍익대학교 _제2캠퍼스,7961,441,68,64,288,11,0,0,...,13.02,0,0,0,0,0,0,0,0,0
243,2022,홍익대학교,17235,628,67,70,434,35,0,0,...,9.77,0,0,0,0,0,0,0,0,0
244,2022,화성의과학대학교,586,53,4,15,34,0,0,0,...,14.08,0,0,0,0,0,0,0,0,0


In [800]:
df2

,기준연도,학교,외국 재적학생 총계(A),중도탈락 학생 총계(B),학위과정(대학),교육과정공동운영생,어학연수생,교환학생,방문학생,기타연수생,외국 학생 중도탈락률(%)(B/A)×100
2,2022,가야대학교(김해),21,2,2,0,0,0,0,0,9.5
3,2022,가천대학교,2131,140,140,0,0,0,0,0,6.6
4,2022,가톨릭관동대학교,49,6,5,0,1,0,0,0,12.2
5,2022,가톨릭대학교,499,22,22,0,0,0,0,0,4.4
6,2022,가톨릭대학교 _제2캠퍼스,1,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
219,2022,호서대학교,355,34,22,0,12,0,0,0,9.6
220,2022,호원대학교,173,17,17,0,0,0,0,0,9.8
221,2022,홍익대학교,1668,115,75,0,40,0,0,0,6.9
222,2022,홍익대학교 _제2캠퍼스,110,21,21,0,0,0,0,0,19.1


## 2. 연도+학교명 키값으로 두개의 df를 결합

- df2의 첫줄을 읽음
- df2key = 기준연도 + 학교명
- df1에서 df1key와 일치하는 레코드 확인
- 있다면.. df2의 특정 필드값을 읽어서 채움

In [801]:
print(df1.columns)
print(df2.columns)

Index(['기준연도', '학교', '재적학생\n(A)', '사유별 중도탈락 학생 계(B)', '미등록', '미복학', '자퇴',
       '학사경고', '학생활동', '유급제적', '재학연한초과', '기타', '중도탈락학생비율(%)\n(B/A) × 100',
       '재적학생\n(신입생)\n(A`)', '사유별 중도탈락 학생(신입생) 계(B`)', '미등록', '미복학', '자퇴',
       '학사경고', '학생활동', '유급제적', '재학연한초과', '기타',
       '중도탈락학생(신입생)비율(%)\n(B`/A`) × 100', '외국 재적학생 총계(A)', '외국 중도탈락 학생 총계(B)',
       '외국 학위과정(대학)', '외국 교육과정공동운영생', '외국 어학연수생', '외국 교환학생', '외국 방문학생',
       '외국 기타연수생', '외국 학생 중도탈락률(%)(B/A)×100'],
      dtype='object')
Index(['기준연도', '학교', '외국 재적학생 총계(A)', '중도탈락 학생 총계(B)', '학위과정(대학)', '교육과정공동운영생',
       '어학연수생', '교환학생', '방문학생', '기타연수생', '외국 학생 중도탈락률(%)(B/A)×100'],
      dtype='object')


In [802]:
#print(df3.loc[2])  # 첫번째 행 선택
#print(df3.loc[2, '기준연도'])  # 첫번째 행의 'Name' 열 선택
#print(df3.loc[2, '학교'])  # 첫번째 행의 'Name' 열 선택

df1key = str(df1.loc[2, '기준연도']) + df1.loc[2, '학교'] #전처리중 인덱드2줄을 삭제했기 때문에...
df2key = str(df2.loc[2, '기준연도']) + df2.loc[2, '학교']

print('df1key : ', df1key)
print('df2key : ', df2key)


df1key :  2022가야대학교(김해)
df2key :  2022가야대학교(김해)


In [803]:
for m in range(len(df2)): #df2의 키와 일치하는 값을 찾음
    df2key = str(df2.loc[2+m, '기준연도']) + df2.loc[2+m, '학교']
    #print('df2key : ', df2key)

    for n in range(len(df1)): #추가할 필드만큼만 루프
        df1key = str(df1.loc[2+n, '기준연도']) + df1.loc[2+n, '학교']
        #print('df1key : ', df1key)
        
        if df1key == df2key:
            #print('df1key : ', df1key)
            #print('df2key : ', df2key)
            
            df1list = list(df1.loc[2+n]) #df1은 24개
            df2list = list(df2.loc[2+m]) #df2는 9개
            
            #print('df1list : ', df1list)
            #print('df2list : ', df2list)
            
            
            dfadd_data = df1list[0:24] + df2list[2:]
            #print('dfadd_data : ', dfadd_data)
            
            df1.loc[2+n] = dfadd_data
            
            
            break #같은 키를 찾았으니 브레이크
            
       

C:\Users\acery\AppData\Local\Temp\ipykernel_14296\2218588485.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '9.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df1.loc[2+n] = dfadd_data


In [804]:
df1

,기준연도,학교,재적학생\n(A),사유별 중도탈락 학생 계(B),미등록,미복학,자퇴,학사경고,학생활동,유급제적,...,중도탈락학생(신입생)비율(%)\n(B`/A`) × 100,외국 재적학생 총계(A),외국 중도탈락 학생 총계(B),외국 학위과정(대학),외국 교육과정공동운영생,외국 어학연수생,외국 교환학생,외국 방문학생,외국 기타연수생,외국 학생 중도탈락률(%)(B/A)×100
2,2022,가야대학교(김해),2355,218,29,50,139,0,0,0,...,9.40,21,2,2,0,0,0,0,0,9.5
3,2022,가천대학교,26126,945,125,162,650,0,0,0,...,7.06,2131,140,140,0,0,0,0,0,6.6
4,2022,가톨릭관동대학교,10959,850,318,0,531,0,0,0,...,10.04,49,6,5,0,1,0,0,0,12.2
5,2022,가톨릭꽃동네대학교,545,24,5,0,19,0,0,0,...,6.48,0,0,0,0,0,0,0,0,0.0
6,2022,가톨릭대학교 _제3캠퍼스,306,6,1,0,4,1,0,0,...,2.04,6,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,2022,호원대학교,6644,745,209,177,358,0,0,0,...,15.33,173,17,17,0,0,0,0,0,9.8
242,2022,홍익대학교 _제2캠퍼스,7961,441,68,64,288,11,0,0,...,13.02,110,21,21,0,0,0,0,0,19.1
243,2022,홍익대학교,17235,628,67,70,434,35,0,0,...,9.77,1668,115,75,0,40,0,0,0,6.9
244,2022,화성의과학대학교,586,53,4,15,34,0,0,0,...,14.08,0,0,0,0,0,0,0,0,0.0


### 학교명을 코드명으로 변경
- 학교명과 일치하는 univ_cd를 찾아서 데이터프레임 정리

In [805]:


# db truncate tables
# 기존 데이터를 삭제 후 인서트, 계속 추가하는 로직이면 삭제
# sql_truncate = 'truncate table {}.{}'.format(dbname, table_dropout)

# curs.execute(sql_truncate)
# conn.commit()

In [806]:
# university 테이블에서 학교명과 일치하는 univ_cd를 찾음

field_names = ''
field_data = ''


sql_select = 'SELECT {}, {} FROM {}'.format('univ_cd', 'univ_name_arlimi', table_univ)
curs.execute(sql_select)

univ_data = curs.fetchall()


print(type(univ_data), univ_data)
print(type(univ_data[0]), univ_data[0])
print(type(univ_data[0]['univ_name_arlimi']), univ_data[0]['univ_name_arlimi'], univ_data[0]['univ_cd'])
#conn.commit()


<class 'list'> [{'univ_cd': '0000001', 'univ_name_arlimi': '국립강릉원주대학교'}, {'univ_cd': '0000002', 'univ_name_arlimi': '국립강릉원주대학교 _제2캠퍼스'}, {'univ_cd': '0000003', 'univ_name_arlimi': '강원대학교'}, {'univ_cd': '0000004', 'univ_name_arlimi': '강원대학교 _제2캠퍼스'}, {'univ_cd': '0000005', 'univ_name_arlimi': '경북대학교'}, {'univ_cd': '0000007', 'univ_name_arlimi': '경상국립대학교'}, {'univ_cd': '0000008', 'univ_name_arlimi': '국립공주대학교'}, {'univ_cd': '0000009', 'univ_name_arlimi': '국립군산대학교'}, {'univ_cd': '0000010', 'univ_name_arlimi': '국립금오공과대학교'}, {'univ_cd': '0000011', 'univ_name_arlimi': '국립목포대학교'}, {'univ_cd': '0000012', 'univ_name_arlimi': '국립목포해양대학교'}, {'univ_cd': '0000013', 'univ_name_arlimi': '국립부경대학교'}, {'univ_cd': '0000014', 'univ_name_arlimi': '부산대학교'}, {'univ_cd': '0000019', 'univ_name_arlimi': '서울대학교'}, {'univ_cd': '0000020', 'univ_name_arlimi': '국립순천대학교'}, {'univ_cd': '0000021', 'univ_name_arlimi': '국립안동대학교'}, {'univ_cd': '0000023', 'univ_name_arlimi': '전남대학교'}, {'univ_cd': '0000024', 'univ_name_arlim

In [807]:
for n in range(len(df1)): #df1에 통합된 중도 탈락학생 데이터프레임에서 학교명을 코드명으로 변경
    df1_univname = str(df1.loc[2+n, '학교'])
    idx = 2+n
    #print(type(df1uname), df1uname, univ_data[0]['univ_name_arlimi'])
    #print(len(univ_data))
    
    for i in range(len(univ_data)): #466
        if df1_univname == univ_data[i]['univ_name_arlimi']:
            #print(univ_data[i]['univ_name_arlimi'], univ_data[i]['univ_cd'])
            print(df1_univname, univ_data[i]['univ_name_arlimi'], univ_data[i]['univ_cd'])
            
            df1.loc[idx, '학교'] = univ_data[i]['univ_cd']
            break
            

가야대학교(김해) 가야대학교(김해) 0002940
가천대학교 가천대학교 0000063
가톨릭관동대학교 가톨릭관동대학교 0000072
가톨릭꽃동네대학교 가톨릭꽃동네대학교 1000001
가톨릭대학교 _제3캠퍼스 가톨릭대학교 _제3캠퍼스 0000049
가톨릭대학교 _제2캠퍼스 가톨릭대학교 _제2캠퍼스 0000048
가톨릭대학교 가톨릭대학교 0000046
감리교신학대학교 감리교신학대학교 0000050
강남대학교 강남대학교 0000051
국립강릉원주대학교 _제2캠퍼스 국립강릉원주대학교 _제2캠퍼스 0000002
국립강릉원주대학교 국립강릉원주대학교 0000001
강서대학교 강서대학교 1000002
강원대학교 강원대학교 0000003
강원대학교 _제2캠퍼스 강원대학교 _제2캠퍼스 0000004
건국대학교 건국대학교 0000052
건국대학교(글로컬) _분교 건국대학교(글로컬) _분교 0000053
건양대학교 건양대학교 0000054
건양대학교 _제2캠퍼스 건양대학교 _제2캠퍼스 0000055
건양사이버대학교 건양사이버대학교 0000335
경기대학교 경기대학교 0000056
경기대학교 _제2캠퍼스 경기대학교 _제2캠퍼스 0000058
경남과학기술대학교 경남과학기술대학교 0000035
경남대학교 경남대학교 0000059
경동대학교 _제3캠퍼스 경동대학교 _제3캠퍼스 0002574
경동대학교 _제4캠퍼스 경동대학교 _제4캠퍼스 0002744
경동대학교 경동대학교 0000060
경북대학교 경북대학교 0000005
경상국립대학교 경상국립대학교 0000007
경성대학교 경성대학교 0000062
경운대학교 경운대학교 0000244
경인교육대학교 경인교육대학교 0000256
경인교육대학교 _제2캠퍼스 경인교육대학교 _제2캠퍼스 0000257
경일대학교 경일대학교 0000064
경주대학교 경주대학교 0000065
경희대학교 경희대학교 0000066
경희사이버대학교 경희사이버대학교 0000318
계명대학교 계명대학교 0000068
고려대학교 고려대학교 0000069
고려대학교(세종) _분교 

In [808]:
df1

,기준연도,학교,재적학생\n(A),사유별 중도탈락 학생 계(B),미등록,미복학,자퇴,학사경고,학생활동,유급제적,...,중도탈락학생(신입생)비율(%)\n(B`/A`) × 100,외국 재적학생 총계(A),외국 중도탈락 학생 총계(B),외국 학위과정(대학),외국 교육과정공동운영생,외국 어학연수생,외국 교환학생,외국 방문학생,외국 기타연수생,외국 학생 중도탈락률(%)(B/A)×100
2,2022,0002940,2355,218,29,50,139,0,0,0,...,9.40,21,2,2,0,0,0,0,0,9.5
3,2022,0000063,26126,945,125,162,650,0,0,0,...,7.06,2131,140,140,0,0,0,0,0,6.6
4,2022,0000072,10959,850,318,0,531,0,0,0,...,10.04,49,6,5,0,1,0,0,0,12.2
5,2022,1000001,545,24,5,0,19,0,0,0,...,6.48,0,0,0,0,0,0,0,0,0.0
6,2022,0000049,306,6,1,0,4,1,0,0,...,2.04,6,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,2022,0000282,6644,745,209,177,358,0,0,0,...,15.33,173,17,17,0,0,0,0,0,9.8
242,2022,0002949,7961,441,68,64,288,11,0,0,...,13.02,110,21,21,0,0,0,0,0,19.1
243,2022,0000212,17235,628,67,70,434,35,0,0,...,9.77,1668,115,75,0,40,0,0,0,6.9
244,2022,1000006,586,53,4,15,34,0,0,0,...,14.08,0,0,0,0,0,0,0,0,0.0


### 전처리 완료된 중도탈락 데이터를 DB에 인서트
- dbname = 'cosshubtest'  ## 변경 필요
- table_univ = 'university'    ## 변경 필요
- table_dropout = 'dropout_stat'    ## 변경 필요

In [809]:
# db truncate tables
# 기존 데이터를 삭제 후 인서트, 계속 추가하는 로직이면 삭제
sql_truncate = 'truncate table {}.{}'.format(dbname, table_dropout)

curs.execute(sql_truncate)
conn.commit()


year, univ_cd, stud_tot, stud_sum, stud_unregist, stud_unreturn, stud_drop, stud_warn, stud_act, stud_expulsion, stud_attendyear, stud_etc, stud_rate, fresh_tot, fresh_sum, fresh_unregist, fresh_unreturn, fresh_drop, fresh_warn, fresh_act, fresh_expulsion, fresh_attendyear, fresh_etc, fresh_rate, foreign_tot, foreign_sum, foreign_degree, foreign_common, foreign_lang, foreign_exchange, foreign_visit, foreign_etc, foreign_rate

In [831]:
# data insert to db

field_names = ''
field_data_str = ''

drop_columns = len(df1.columns) #엑셀 df읠 필드명

sql_col = 'SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = "{}"'.format(table_dropout)

curs.execute(sql_col)
#conn.commit()
col_name = curs.fetchall() #컬럼명, 리스트 안에 딕셔너리로...

col_name_list = list()
col_name_str = ''
for n in range(len(col_name)):
    col_name_list.append(col_name[n]['COLUMN_NAME'])
    
for n in range(len(col_name)):
    if n == 0:
        col_name_str = str(col_name_list[n]) 
    else:
        col_name_str = col_name_str + ', ' + str(col_name_list[n])
        

#print(col_name_str)
# print(type(col_name), len(col_name), col_name)
# print(type(col_name[0]), len(col_name[0]), col_name[0])
# print(type(col_name[0]['COLUMN_NAME']), col_name[0]['COLUMN_NAME'])


for n in range(len(df1)): #전처리 완료된 데이터 프레임 전체를 읽어서 DB에 인서트
    if n >= 2 : #첫 두줄은 제목행임
        field_data = list(df1.iloc[n]) # 시리즈 -> 리스트
        #print(field_data)
        #print(field_data[0])
        
        for m in range(len(col_name)):
            if m == 0: #2023
                field_data_str = str(field_data[m])            
            elif m == 1: # "0000069"
                field_data_str = field_data_str + ', "' + str(field_data[m] + '"')
            else:
                field_data_str = field_data_str + ', ' + str(field_data[m]) 
        
        print(type(field_data_str), field_data_str)
        sql_insert = 'insert into {}.{}({}) values({})'.format(dbname, table_dropout, col_name_str, field_data_str)
        print(n, field_data[1], sql_insert)
        curs.execute(sql_insert)
        #print(col_name_str)
        #print(field_data_str)
        
conn.commit()

# sql_insert = 'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
# print(sql_insert)


#print(sql)
# for idx in range(len(df)):
#     	curs.execute(sql_insert, tuple(df.values[idx]))
# conn.commit()


<class 'str'> 2022, "0000072", 10959, 850, 318, 0, 531, 0, 0, 0, 0, 1, 7.76, 1563, 157, 3, 0, 153, 0, 0, 0, 0, 1, 10.04, 49, 6, 5, 0, 1, 0, 0, 0, 12.2
2 0000072 insert into cosshubtest.dropout_stat(year, univ_cd, stud_tot, stud_sum, stud_unregist, stud_unreturn, stud_drop, stud_warn, stud_act, stud_expulsion, stud_attendyear, stud_etc, stud_rate, fresh_tot, fresh_sum, fresh_unregist, fresh_unreturn, fresh_drop, fresh_warn, fresh_act, fresh_expulsion, fresh_attendyear, fresh_etc, fresh_rate, foreign_tot, foreign_sum, foreign_degree, foreign_common, foreign_lang, foreign_exchange, foreign_visit, foreign_etc, foreign_rate) values(2022, "0000072", 10959, 850, 318, 0, 531, 0, 0, 0, 0, 1, 7.76, 1563, 157, 3, 0, 153, 0, 0, 0, 0, 1, 10.04, 49, 6, 5, 0, 1, 0, 0, 0, 12.2)
<class 'str'> 2022, "1000001", 545, 24, 5, 0, 19, 0, 0, 0, 0, 0, 4.4, 108, 7, 0, 0, 7, 0, 0, 0, 0, 0, 6.48, 0, 0, 0, 0, 0, 0, 0, 0, 0.0
3 1000001 insert into cosshubtest.dropout_stat(year, univ_cd, stud_tot, stud_sum, stud_unre

## 이상 uiversity와 dropout_stat 테이블 생성 완료